In [1]:
# Read json file
import json
import numpy as np
import requests
import os
import pandas as pd
from sklearn import preprocessing
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import warnings

# Dataframe

In [2]:
playlists = json.load(open('playlists.json', 'r'))
tracks = json.load(open('track_features.json', 'r'))

In [3]:
playlists_df = pd.read_csv('playlists.csv')
playlists_df = playlists_df.drop(['key'], axis=1)

# Train test split
np.random.seed(42)
msk = np.random.rand(len(playlists_df)) < 0.8
playlists_df, test_playlists_df = playlists_df[msk], playlists_df[~msk]

headers = playlists_df.columns.values.tolist()

print(headers)

mean = playlists_df[headers[2:]].mean(axis=0)
std = playlists_df[headers[2:]].std(axis=0)

playlists_df[headers[2:]] = (playlists_df[headers[2:]] - mean) / std
test_playlists_df[headers[2:]] = (test_playlists_df[headers[2:]] - mean) / std

cluster_headers = headers[2:]

['index', 'title', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']


In [4]:
# playlists_df = pd.read_csv('playlists_embeddings.csv')
# cols = ['danceability', 'energy', 'loudness', 'mode', 'speechiness',
#         'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
#         'duration_ms', 'time_signature']
# playlists_df = playlists_df.drop(cols + ['key'], axis=1)

# # Train test split
# np.random.seed(42)
# msk = np.random.rand(len(playlists_df)) < 0.8
# playlists_df, test_playlists_df = playlists_df[msk], playlists_df[~msk]

# headers = playlists_df.columns.values.tolist()

# print(headers)

# mean = playlists_df[headers[2:]].mean(axis=0)
# std = playlists_df[headers[2:]].std(axis=0)

# playlists_df[headers[2:]] = (playlists_df[headers[2:]] - mean) / std
# test_playlists_df[headers[2:]] = (test_playlists_df[headers[2:]] - mean) / std

# cluster_headers = headers[2:]

# Clustering

### K-means

In [5]:
def get_cluster_tracks(playlists_df, cluster_preds, num_clusters, verbose=False):
    global playlists, tracks
    
    playlists_clustered = playlists_df.get(['index', 'title']).copy()
    playlists_clustered['cluster'] = cluster_preds

    cluster_tracks = [{} for _ in range(num_clusters)]
    for cluster in range(num_clusters):
        for i in playlists_clustered[playlists_clustered['cluster'] == cluster]['index']:
            for track in playlists[i][1]:
                if track['track_uri'] not in cluster_tracks[cluster]:
                    cluster_tracks[cluster][track['track_uri']] = track
        cluster_tracks[cluster] = np.array(list(cluster_tracks[cluster].values()))

        if verbose:
            print('Cluster {}: {} tracks'.format(cluster, len(cluster_tracks[cluster])))

    return cluster_tracks

def cluster_custom_score(playlists_df, cluster_preds, num_clusters):
    global tracks
    
    cluster_tracks = get_cluster_tracks(playlists_df, cluster_preds, num_clusters)

    lengths = np.array([len(tracks) for tracks in cluster_tracks])

    return np.sum(lengths) / len(tracks), \
        np.max(lengths) / len(tracks), \
        np.std(lengths) / np.mean(lengths)

In [6]:
NUM_CLUSTERS = 50
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init='auto').fit(playlists_df[headers[2:]])

In [7]:
# Group playlists by cluster
playlists_df['cluster'] = kmeans.labels_
playlists_clustered = playlists_df.get(['index', 'title', 'cluster'])

In [8]:
# Count most common titles
def count_words(titles):
    words = []
    for title in titles:
        title = str(title).strip().lower()
        words += title.split(" ")
    return Counter(words)


for i in range(NUM_CLUSTERS):
    print(count_words(
        playlists_clustered[playlists_clustered['cluster'] == i]['title'].values.tolist()).most_common(10))

[('summer', 98), ('2017', 47), ('2016', 43), ('party', 42), ('music', 42), ('good', 40), ('new', 37), ('chill', 37), ('songs', 34), ('playlist', 34)]
[('chill', 39), ('sleep', 21), ('the', 17), ('feels', 13), ('songs', 13), ('sad', 12), ('calm', 11), ('music', 10), ('mellow', 10), ('christmas', 8)]
[('chill', 40), ('songs', 17), ('love', 17), ('good', 16), ('wedding', 14), ('new', 14), ('the', 12), ('summer', 12), ('playlist', 11), ('country', 11)]
[('classical', 25), ('study', 12), ('music', 11), ('instrumental', 10), ('scores', 6), ('movie', 5), ('piano', 5), ('the', 5), ('of', 4), ('studying', 3)]
[('rap', 95), ('workout', 57), ('party', 34), ('hop', 29), ('hype', 27), ('hip', 26), ('gym', 24), ('music', 23), ('up', 22), ('lit', 17)]
[('rock', 46), ('punk', 38), ('pop', 23), ('my', 15), ('workout', 13), ('angst', 13), ('country', 12), ('songs', 10), ('running', 9), ('music', 9)]
[('rock', 131), ('classic', 34), ("90's", 19), ('the', 17), ('90s', 15), ('music', 14), ('good', 12), ('a

# Test data

In [9]:
test_playlists_df['cluster'] = kmeans.predict(test_playlists_df[headers[2:]])
test_playlists_clustered = test_playlists_df.get(['index', 'title', 'cluster'])

for i in range(NUM_CLUSTERS):
    print(count_words(
        test_playlists_clustered[test_playlists_clustered['cluster'] == i]['title'].values.tolist()).most_common(10))

[('summer', 23), ('vibes', 12), ('new', 11), ('my', 11), ('songs', 11), ('2016', 10), ('good', 9), ('2017', 9), ('workout', 8), ('music', 7)]
[('chill', 12), ('sleep', 6), ('sad', 5), ('songs', 5), ('calm', 5), ('acoustic', 4), ('feels', 4), ('new', 4), ('yoga', 4), ('music', 3)]
[('chill', 18), ('wedding', 8), ('the', 4), ('good', 4), ('cocktail', 3), ('hour', 3), ('mix', 3), ('fall', 3), ('playlist', 3), ('stuff', 2)]
[('study', 6), ('piano', 5), ('music', 4), ('soundtracks', 3), ('classical', 3), ('movie', 2), ('relaxation', 2), ('instrumental', 2), ('time', 2), ('yo', 2)]
[('rap', 29), ('workout', 15), ('party', 12), ('up', 9), ('playlist', 6), ('lit', 6), ('game', 6), ('music', 5), ('jams', 5), ('hip', 5)]
[('punk', 10), ('workout', 9), ('rock', 4), ('my', 4), ('high-intensity', 4), ('throwback', 4), ('music', 3), ('good', 3), ('for', 3), ('pop', 3)]
[('rock', 24), ('90s', 6), ('alternative', 5), ('playlist', 5), ('the', 5), ('stuff', 4), ('music', 3), ('songs', 3), ('of', 3), ('c

# Get recommendations (for test playlists)

In [10]:
track_popularity = json.load(open('track_popularity.json', 'r'))

K-nn is always based on Spotify features

In [11]:
playlists_clusters = playlists_df['cluster'].copy()
test_playlists_clusters = test_playlists_df['cluster'].copy()

playlists_df = pd.read_csv('playlists_popularity.csv')
playlists_df = playlists_df.drop(['key'], axis=1)
cols = ['danceability', 'energy', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'time_signature']
playlists_df, test_playlists_df = playlists_df[msk], playlists_df[~msk]

headers = playlists_df.columns.values.tolist()

print(headers)

mean = playlists_df[headers[2:]].mean(axis=0)
std = playlists_df[headers[2:]].std(axis=0)

playlists_df[headers[2:]] = (playlists_df[headers[2:]] - mean) / std
test_playlists_df[headers[2:]] = (test_playlists_df[headers[2:]] - mean) / std

playlists_df = pd.concat((playlists_df, playlists_clusters), axis=1)
test_playlists_df = pd.concat((test_playlists_df, test_playlists_clusters), axis=1)

['index', 'title', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'popularity']


In [12]:
mean = mean[:len(cols)]
std = std[:len(cols)]

The challenge for the **input** only withholds a constant number of tracks. For simplicity, **we just withhold half the playlist.**

The challenge **output** requires a **list of 500 recommended candidate tracks**, ordered by relevance in decreasing order. We omit the ordering since we do not evaluate the ordering.

In [13]:
cluster_tracks = get_cluster_tracks(playlists_df, playlists_df['cluster'], NUM_CLUSTERS)

PART_PERCENT = 0.5 # Percentage of playlist to use for clustering

In [14]:
def get_playlist_features(playlist_tracks, mean=mean, std=std):
    features = [np.mean([track[col] for track in playlist_tracks]) for col in cols]
    return np.array((features - mean) / std)

def get_track_info(tracks):
    return np.array([[s['artist_name'], s['track_name']] for s in tracks])

def get_track_features(tracks, mean=mean, std=std, hashmap=False):
    if hashmap:
        features = {uri: [tracks[uri][col] for col in cols] for uri in tracks}
        return {uri: np.array((f - mean) / std) for uri, f in features.items()}

    features = np.array([[s[col] for col in cols] for s in tracks])
    return np.array((features - np.array(mean)) / np.array(std))

In [15]:
def k_nn(k, needle_features, haystack_features):
    """
    Given an instance of features, find the k nearest neighbors in the haystack. 
    Return indexes within haystack.
    """
    distances = np.linalg.norm(needle_features - haystack_features, axis=1)
    return np.argsort(distances)[:k]

In [16]:
def r_precision(pred_tracks: set, target_tracks: set):
    return len(pred_tracks.intersection(target_tracks)) / len(target_tracks)

In [17]:
def popularity(pred_tracks: set):
    return np.mean([track_popularity[track] for track in pred_tracks])

K-nn to find nearest playlists (then randomly sample tracks)

In [18]:
def nearest_playlists_predictions(init_tracks, target_tracks, playlist_part_features, part_cluster):
    pred_tracks = init_tracks.copy()
    
    haystack_playlists = playlists_df[playlists_df['cluster'] == part_cluster] 
    haystack_features = haystack_playlists[cols].values

    for i in k_nn(500 + len(init_tracks), playlist_part_features, haystack_features):
        pred_playlist = haystack_playlists.iloc[i]
        i = pred_playlist['index']
        
        for uri in [track['track_uri'] for track in playlists[i][1]]:
            pred_tracks.add(uri)

            if len(pred_tracks) >= 500 + len(init_tracks):
                break

        if len(pred_tracks) >= 500 + len(init_tracks):
            break

    pred_tracks = pred_tracks - init_tracks

    return r_precision(pred_tracks, target_tracks), popularity(pred_tracks)

K-NN to find tracks nearest to playlist aggregate features

In [19]:
def nearest_aggregate_predictions(init_tracks, target_tracks, playlist_part_features, tracks_in_cluster, track_features):
    pred_tracks = init_tracks.copy()
    
    for i in k_nn(500 + len(init_tracks), playlist_part_features, track_features):
        pred_tracks.add(tracks_in_cluster[i]['track_uri'])

        if len(pred_tracks) >= 500 + len(init_tracks):
            break

    pred_tracks = pred_tracks - init_tracks

    return r_precision(pred_tracks, target_tracks), popularity(pred_tracks)

K-NN to find tracks nearest to random tracks in playlist

In [20]:
def nearest_track_predictions(init_tracks, target_tracks, playlist_part_tracks, tracks_in_cluster, track_features, all_track_features_dict):
    pred_tracks = init_tracks.copy()
    
    playlist_track_features = [all_track_features_dict[track['track_uri']] for track in playlist_part_tracks]
    playlist_track_nn = [k_nn(len(track_features), features, track_features) for features in playlist_track_features] # bottleneck

    for i in range(500):
        for track_nn in playlist_track_nn:
            if i >= len(track_nn):
                continue

            pred_tracks.add(tracks_in_cluster[track_nn[i]]['track_uri'])

            if len(pred_tracks) >= 500 + len(init_tracks):
                break

        if len(pred_tracks) >= 500 + len(init_tracks):
            break

    pred_tracks = pred_tracks - init_tracks

    return r_precision(pred_tracks, target_tracks), popularity(pred_tracks)

In [21]:
cluster_track_features = [get_track_features(cluster_tracks[i]) for i in range(NUM_CLUSTERS)]
all_track_features_dict = get_track_features(tracks, hashmap=True)

In [22]:
playlists_predictions_score = 0
aggregate_predictions_score = 0
track_predictions_score = 0
playlists_predictions_popularity = 0
aggregate_predictions_popularity = 0
track_predictions_popularity = 0

num_test = len(test_playlists_df)

for test_i in range(num_test):

    i = test_playlists_df.iloc[test_i]['index']

    playlist_name = playlists[i][0]
    playlist_tracks = playlists[i][1]
    np.random.shuffle(playlist_tracks)

    playlist_part_tracks = playlist_tracks[:int(
        len(playlists[i][1]) * PART_PERCENT)]

    playlist_part_features = get_playlist_features(playlist_part_tracks)

    playlist_features = np.array(
        test_playlists_df.iloc[test_i][headers[2:]].values, dtype='float32')

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        part_cluster = kmeans.predict([test_playlists_df.iloc[test_i][cluster_headers]])[0]

    tracks_in_cluster = cluster_tracks[part_cluster]
    track_features = cluster_track_features[part_cluster]

    init_tracks = set([track['track_uri'] for track in playlist_part_tracks])
    target_tracks = set([track['track_uri']
                        for track in playlist_tracks]) - init_tracks

    t1, t2 = nearest_playlists_predictions(
        init_tracks, target_tracks, playlist_part_features, part_cluster)
    playlists_predictions_score += t1
    playlists_predictions_popularity += t2

    t1, t2 = nearest_aggregate_predictions(
        init_tracks, target_tracks, playlist_part_features, tracks_in_cluster, track_features)
    aggregate_predictions_score += t1
    aggregate_predictions_popularity += t2

    t1, t2 = nearest_track_predictions(
        init_tracks, target_tracks, playlist_part_tracks, tracks_in_cluster, track_features, all_track_features_dict)
    track_predictions_score += t1
    track_predictions_popularity += t2
    
print(f'R-precision: {100 * playlists_predictions_score / num_test:.2f}%, popularity: {playlists_predictions_popularity / num_test:.2f}')
print(f'R-precision: {100 * aggregate_predictions_score / num_test:.2f}%, popularity: {aggregate_predictions_popularity / num_test:.2f}')
print(f'R-precision: {100 * track_predictions_score / num_test:.2f}%, popularity: {track_predictions_popularity / num_test:.2f}')



R-precision: 22.83%, popularity: 34.70
R-precision: 3.92%, popularity: 25.68
R-precision: 4.65%, popularity: 26.50
